In [1]:
!pip install torch torchvision
!git clone https://github.com/neubig/nn4nlp-code.git

fatal: destination path 'nn4nlp-code' already exists and is not an empty directory.


In [0]:
from collections import defaultdict
import time
import random
import torch
import torch.nn as nn
import numpy as np

In [0]:
# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      tag, words = line.lower().strip().split(" ||| ")
      yield ([w2i[x] for x in words.split(" ")], t2i[tag])

# Read in the data
train = list(read_dataset("nn4nlp-code/data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("nn4nlp-code/data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

In [0]:
class Model(nn.Module):
  def __init__(self, emb_size, hid_size, hid_lay):
    super(Model, self).__init__()
    self.embeddings = nn.Embedding(nwords, emb_size)
    linears = []
    for lay in range(hid_lay-1):
      in_dim = emb_size if lay == 0 else hid_size
      linears += [nn.Linear(in_dim, hid_size), nn.Tanh()]
    linears += [nn.Linear(emb_size, ntags)]
    self.fc = nn.Sequential(*linears)
  
  def forward(self, x):
    embed_out = self.embeddings(x)
    score = self.fc(embed_out.sum(0))
    return score

In [0]:
model = Model(64, 64, 2)
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters())

In [58]:
model, len(train)

(Model(
   (embeddings): Embedding(18648, 64)
   (fc): Sequential(
     (0): Linear(in_features=64, out_features=64, bias=True)
     (1): Tanh()
     (2): Linear(in_features=64, out_features=5, bias=True)
   )
 ), 8544)

In [0]:
for ITER in range(100):
  random.shuffle(train)
  train_loss = 0.0
  start = time.time()
  model.train(True)
  for words, tag in train:
    words = torch.LongTensor(words)
    tag = torch.LongTensor([tag])
    model.zero_grad()
    
    logits = model(words)
#     print(logits.shape)
#     print(tag.shape)
    my_loss = loss_fn(logits.view(-1, 5), tag)
#     my_loss = loss_fn(logits, tag)
    train_loss += my_loss.data
    my_loss.backward()
    optim.step()
  print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time() -start))
  
  test_correct = 0.0
  model.train(False)
  for words, tag in dev:
    words = torch.LongTensor(words)
    tag = torch.LongTensor([tag])
    logits = model(words)
    predict = torch.argmax(logits)
    if predict == tag:
      test_correct += 1
  print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))

iter 0: train loss/sent=1.5597, time=378.98s
iter 0: test acc=0.3009
iter 1: train loss/sent=1.3838, time=598.43s
iter 1: test acc=0.3457
iter 2: train loss/sent=1.1909, time=598.18s
iter 2: test acc=0.3665
iter 3: train loss/sent=1.0108, time=598.24s
iter 3: test acc=0.3652
iter 4: train loss/sent=0.8311, time=598.39s
iter 4: test acc=0.3910
iter 5: train loss/sent=0.6522, time=598.92s
iter 5: test acc=0.3507
iter 6: train loss/sent=0.4986, time=598.76s
iter 6: test acc=0.3566
iter 7: train loss/sent=0.3696, time=599.85s
iter 7: test acc=0.3570
iter 8: train loss/sent=0.2684, time=600.14s
iter 8: test acc=0.3665
iter 9: train loss/sent=0.1901, time=600.49s
iter 9: test acc=0.3570
iter 10: train loss/sent=0.1319, time=601.46s
iter 10: test acc=0.3575
iter 11: train loss/sent=0.1011, time=602.08s
iter 11: test acc=0.3543
iter 12: train loss/sent=0.0715, time=603.08s
iter 12: test acc=0.3593
iter 13: train loss/sent=0.0536, time=604.51s
iter 13: test acc=0.3548
iter 14: train loss/sent=0